## PySpark - Recomendadores - Filtrado Colaborativo

In [ ]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("recomendador").getOrCreate()

In [3]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

### Alternating Least Squares - ALS

ALS se utiliza para resolver el problema de la factorización matricial en sistemas de recomendación.
El objetivo de la factorización matricial en este contexto es encontrar dos matrices, una de **usuarios** y otra de **ítems**, que cuando se multiplican entre sí, aproximan la matriz de valoraciones (o interacciones) entre usuarios e ítems.

El algoritmo ALS busca encontrar estas dos matrices de manera iterativa, y lo hace a través de dos pasos que se alternan en cada iteración:

- Fijar la matriz de usuarios y resolver para la matriz de ítems. En este paso, se utiliza la regresión lineal para ajustar una matriz de ítems que pueda explicar la matriz de valoraciones conocidas. Para hacerlo, se utiliza la matriz de usuarios fija para predecir la matriz de valoraciones que debería haber para cada ítem. Una vez que se tienen estas predicciones, se ajusta la matriz de ítems para que las predicciones se ajusten lo mejor posible a la matriz de valoraciones conocida.

- Fijar la matriz de ítems y resolver para la matriz de usuarios. En este paso, se utiliza el mismo proceso que en el paso anterior, pero ahora se fija la matriz de ítems y se ajusta la matriz de usuarios para que las predicciones de valoraciones se ajusten lo mejor posible a la matriz de valoraciones conocida.

Estos dos pasos se repiten iterativamente hasta que se alcanza un punto de convergencia, es decir, hasta que las matrices de usuarios e ítems que se obtienen en una iteración no son muy diferentes de las obtenidas en la iteración anterior.

Es importante destacar que el algoritmo ALS no es determinista, es decir, que puede arrojar resultados ligeramente diferentes en cada ejecución, ya que en cada iteración se utilizan valores aleatorios para inicializar las matrices de usuarios e ítems.


In [4]:
df = spark.read.csv(path        = "../data/ratings.csv",
                    inferSchema = True,
                    header      = True)

In [5]:
df.show(5, truncate = False)

+------+-------+------+----------+
|userId|movieId|rating|timestamp |
+------+-------+------+----------+
|1     |169    |2.5   |1204927694|
|1     |2471   |3.0   |1204927438|
|1     |48516  |5.0   |1204927435|
|2     |2571   |3.5   |1436165433|
|2     |109487 |4.0   |1436165496|
+------+-------+------+----------+
only showing top 5 rows



In [6]:
# Eliminamos esta columna porque no la necesitamos y ahorramos espacio.

df = df.drop("timestamp")

In [7]:
df.count()

22884377

In [8]:
# Cortamos el df porque no se puede ejecutar por el tamaño en Kaggle.

print(f'Tamaño original: {df.count()}')

df = df.filter(df["userId"] < 10000)

print(f'Tamaño nuevo: {df.filter(df["userId"] < 10000).count()}')

Tamaño original: 22884377
Tamaño nuevo: 925160


In [9]:
df.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|    169|   2.5|
|     1|   2471|   3.0|
|     1|  48516|   5.0|
|     2|   2571|   3.5|
|     2| 109487|   4.0|
|     2| 112552|   5.0|
|     2| 112556|   4.0|
|     3|    356|   4.0|
|     3|   2394|   4.0|
|     3|   2431|   5.0|
|     3|   2445|   4.0|
|     4|     16|   4.0|
|     4|     39|   4.0|
|     4|     45|   4.0|
|     4|     47|   2.0|
|     4|     94|   5.0|
|     4|    101|   4.0|
|     4|    246|   4.0|
|     4|    288|   2.0|
|     4|    296|   4.0|
+------+-------+------+
only showing top 20 rows



In [10]:
df.describe().show()

+-------+------------------+------------------+------------------+
|summary|            userId|           movieId|            rating|
+-------+------------------+------------------+------------------+
|  count|            925160|            925160|            925160|
|   mean| 5003.370057071209|12305.195689394266|3.5001999654113884|
| stddev|2882.6169030661954| 25398.89494107917|1.0738234897489394|
|    min|                 1|                 1|               0.5|
|    max|              9999|            151507|               5.0|
+-------+------------------+------------------+------------------+



In [11]:
train, test = df.randomSplit([0.8, 0.2], seed = 42)

In [12]:
# Eliminamos el df original para ahorrar espacio.

del df

In [13]:
als = ALS(maxIter           = 10,
          regParam          = 0.01,
          userCol           = "userId",
          itemCol           = "movieId",
          ratingCol         = "rating",
          coldStartStrategy = "drop")

model  = als.fit(train)

In [14]:
predictions = model.transform(test)

In [15]:
predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   148|    185|   3.0| 3.0008745|
|   148|    296|   5.0| 3.9802558|
|   148|    592|   3.0|  3.479982|
|   463|    293|   4.0| 4.4105816|
|   463|    296|   4.0| 4.1613636|
|   463|    593|   4.0|  4.106948|
|   463|   1136|   4.0|  4.132324|
|   463|   2028|   4.0|  4.058983|
|   463|   2571|   4.0|  4.166133|
|   471|   1252|   5.0|  4.658882|
|   471|   1293|   5.0| 4.5802517|
|   471|   4886|   4.0| 3.5708985|
|   471|   7153|   3.0| 3.1984956|
|   496|   2858|   4.5|  4.473613|
|   833|    742|   3.5| 5.2569532|
|   833|   4052|   4.0| 3.6657681|
|   833|   4226|   4.5| 4.2410264|
|   833|   6303|   3.0| 3.7094643|
|   833|   7153|   4.0| 3.8569846|
|  1088|    110|   5.0|   3.86134|
+------+-------+------+----------+
only showing top 20 rows



In [16]:
evaluator = RegressionEvaluator(metricName    = "rmse",
                                labelCol      = "rating",
                                predictionCol = "prediction")

rmse = evaluator.evaluate(predictions)

print(f"RMSE: {rmse}")

RMSE: 0.9398505836037273


In [17]:
usuario_activo = test.filter(test["userId"] == 42).select(["movieId", "userId"])

usuario_activo.show()

+-------+------+
|movieId|userId|
+-------+------+
|    783|    42|
|   1197|    42|
|  30810|    42|
|  54259|    42|
+-------+------+



In [18]:
recommendations = model.transform(usuario_activo)

recommendations.orderBy("prediction", ascending = False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|  54259|    42|  4.892921|
|   1197|    42| 4.3784285|
|  30810|    42| 4.2532983|
|    783|    42| 3.5511367|
+-------+------+----------+



In [20]:
train.filter(train['userId']==42).show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|    42|    203|   5.0|
|    42|    585|   0.5|
|    42|    661|   4.5|
|    42|    673|   3.5|
|    42|    720|   5.0|
|    42|    724|   4.0|
|    42|    805|   4.0|
|    42|    914|   5.0|
|    42|   1198|   5.0|
|    42|   1223|   4.0|
|    42|   1271|   4.0|
|    42|   2080|   2.0|
|    42|   2294|   2.0|
|    42|   3033|   3.5|
|    42|   7502|   4.0|
|    42|  31658|   5.0|
|    42|  33004|   5.0|
|    42|  79132|   5.0|
|    42|  83158|   5.0|
+------+-------+------+



In [ ]:
################################################################################################################################